In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

#from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

import torch, torchtext
from torchtext.legacy import data 
import pandas as pd



In [ ]:
import pandas as pd
df = pd.read_csv('/content/datasetSentences.txt',sep="\t", 
                        names=['sentence ids', 'sentence'], skiprows=1)
#sentance_ids = pd.read_csv(os.path.join("/content/datasetSentences.txt"), sep="\t")

In [ ]:
df.shape

(11855, 2)

In [ ]:
df.head()
df.sentence[1]

"The gorgeously elaborate continuation of `` The Lord of the Rings '' trilogy is so huge that a column of words can not adequately describe co-writer\\/director Peter Jackson 's expanded vision of J.R.R. Tolkien 's Middle-earth ."

The code need to have labels and the dictionary 
importing the data labels from "sentiment_labels.txt" file and the dictionary from "dictionary.txt"

In [ ]:
df_dict=pd.read_csv("/content/dictionary.txt",sep="|", names=['phrase', 'phrase ids'])
df_dict.shape

(239232, 2)

In [ ]:
df_dict.head()

,phrase,phrase ids
0,!,0
1,! ',22935
2,! '',18235
3,! Alas,179257
4,! Brilliant,22936


In [ ]:
df_senlbl=pd.read_csv("/content/sentiment_labels.txt",sep="|")
df_senlbl.head()

,phrase ids,sentiment values
0,0,0.50000
1,1,0.50000
2,2,0.44444
3,3,0.50000
4,4,0.42708


divison of classess based on the labels

In [ ]:
def discretize_label(label):
    if label <= 0.2: return 0
    if label <= 0.4: return 1
    if label <= 0.6: return 2
    if label <= 0.8: return 3
    return 4

In [ ]:
df_senlbl['sentiment labels'] = df_senlbl['sentiment values'].apply(discretize_label)
df_senlbl.head()

,phrase ids,sentiment values,sentiment labels
0,0,0.50000,2
1,1,0.50000,2
2,2,0.44444,2
3,3,0.50000,2
4,4,0.42708,2


In [ ]:
sentnc_mrg=pd.merge(df, df_dict, left_on='sentence', right_on='phrase')
df_final = pd.merge(sentnc_mrg, df_senlbl, on='phrase ids')
df_final = df_final[['sentence', 'sentiment labels']]
df_final.head()

,sentence,sentiment labels
0,The Rock is destined to be the 21st Century 's...,3
1,The gorgeously elaborate continuation of `` Th...,4
2,Effective but too-tepid biopic,2
3,If you sometimes like to go to the movies to h...,3
4,"Emerges as something rare , an issue movie tha...",4


In [ ]:
# Import Library
import random
import torch, torchtext
from torchtext import data 

# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [ ]:
#(train, test) = df_final.split(split_ratio=[70, 30], random_state = random.seed(SEED))
#frac=0.7
#df_train = df_final.sample(frac=split, random_state=seed)
#df_test = df_final.loc[~df.index.isin(df_train.index)]

from sklearn.model_selection import train_test_split
trainingSet, testSet = train_test_split(df_final, test_size=0.3)
trainingSet.shape

(7900, 2)

In [ ]:
testSet.shape

(3386, 2)

In [ ]:
Text = torchtext.legacy.data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = torchtext.legacy.data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

In [ ]:
fields = [('texts', Text), ('labels', Label)]

In [ ]:
#train_example = [data.Example.fromlist([df_train['sentence_clean'][i], df_train['sentiment labels'][i]], fields) for i in range(df_train.shape[0])]
#test_example = [data.Example.fromlist([df_test['sentence_clean'][i], df_test['sentiment labels'][i]], fields) for i in range(df_test.shape[0])]
example = [torchtext.legacy.data.Example.fromlist([df_final['sentence'][i], df_final['sentiment labels'][i]], fields) for i in range(df_final.shape[0])]

In [ ]:
dataset_final= torchtext.legacy.data.Dataset(example, fields)

In [ ]:
(train, test) = dataset_final.split(split_ratio=[70, 30], random_state = random.seed(SEED))

In [ ]:
len(train), len(test)

(7900, 3386)

In [ ]:
Text.build_vocab(train)
Label.build_vocab(train)

In [ ]:
print('Size of input vocab : ', len(Text.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Text.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  16406
Size of label vocab :  5
Top 10 words appreared repeatedly : [('.', 7452), (',', 6567), ('the', 5603), ('of', 4061), ('a', 4028), ('and', 4027), ('to', 2822), ('-', 2474), ('is', 2355), ("'s", 2289)]
Labels :  defaultdict(None, {1: 0, 3: 1, 2: 2, 4: 3, 0: 4})


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
train_iterator, test_iterator = torchtext.legacy.data.BucketIterator.splits((train, test), 
                                                           batch_size=128, 
                                                           sort_key=lambda x: len(x.texts),
                                                           sort_within_batch=True,
                                                           device=device)

In [ ]:
next(iter(train_iterator))

RuntimeError: ignored

In [ ]:

import pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Text.vocab.stoi, tokens)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden)   
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output

In [ ]:
# Define hyperparameters
size_of_vocab = len(Text.vocab)
embedding_dim = 300
num_hidden_nodes = 100
num_output_nodes = 3
num_layers = 2
dropout = 0.2

# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

In [ ]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

In [ ]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)


In [ ]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        text, text_lengths = batch.texts 
        
        # convert to 1D tensor
        predictions = model(text, text_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.labels)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.labels)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            text, text_lengths = batch.texts
            
            # convert to 1d tensor
            predictions = model( text, text_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.labels)
            acc = binary_accuracy(predictions, batch.labels)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
N_EPOCHS = 10
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)

     
    # train the model

    
    # evaluate the model
    #valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    #if valid_loss < best_valid_loss:
        #best_valid_loss = valid_loss
        #torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    #print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')